In [83]:
structured_calls = """You are playing a strategic game of trading resources with another player. You are Player 1. You are playing against a player whose resources you have no knowledge about.

You will respond one turn at a time. At each step, you will state your current resources and either propose or respond to a trade.
If it's your turn to propose, your response should be in the following format:

RESOURCES: item1: amount, item2: amount, ... # Your resources

PROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...

and do not say anything else. If it's your turn to respond, your response should be in the following format:

PROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...
PLAYER RESPONSE: {{ACCEPTED/REJECTED}}
RESOURCES AFTER TRADE: item1: amount, item2: amount, ... # Your resources
NEWLY PROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount,  Player 2 Gives: item1: amount, item2: amount, ...

and again, do not say anything else.

Potential Resources in the Game: {}
Your Starting Resources: {}
Your goal: Finish the game with at least {} in under {} rounds of proposals. Otherwise, you will die.

{}
"""
potential_resources = ["X", "Y", "Z"]
potential_resources_txt = ",".join(potential_resources)
starting_resources = {1: {"X": 25, "Y": 5}, 2: {"X": 5, "Y": 25, "Z": 30}}
def resource_str_fn(resources):
    res = [f"{k}: {v}" for k, v in resources.items()]
    return ", ".join(res)

starting_resources_txt = {i: resource_str_fn(starting_resources[i]) for i in starting_resources}

goals = {1: {"X": 15, "Y": 15, "Z": 15}, 2: {"X": 15, "Y": 15, "Z": 15}}
roles = {1: "You start by proposing a trade.", 2: "You start by responding to a trade."}
goals_txt = {i: resource_str_fn(goals[i]) for i in goals}
n_rounds = 4


In [84]:
import openai
import os
import re
from collections import defaultdict

api_key = "None"
os.environ["OPENAI.API_KEY"] = api_key

openai.api_key = os.environ["OPENAI.API_KEY"]

def text_to_dict(s):
    return {k: int(v) for k, v in (item.split(": ") for item in s.split(", "))}

def parse_proposed_trade(s):
    trade = {}
    items = s.split(" Gives:")
    for i in range(1, len(items)):
        item = items[i]
        prev_item = items[i-1]
        player_id = int(prev_item[-2:].strip())
        subitem = item.split(" Player")[0].strip()
        resources = {k: int(v.replace(",", "")) for k, v in (item.split(": ") for item in subitem.split(", "))}
        trade[player_id] = resources
    return trade

def parse_response(response):
    lines = response.split("\n")
    lines_to_pass = defaultdict(list)
    cache = {}
    for l in lines:
        if l.startswith("RESOURCES:"):
            cache["resources"] = [text_to_dict(l.split("RESOURCES: ")[1])]
            
        elif l.startswith("PROPOSED TRADE:"):
            trade = l.split("PROPOSED TRADE:")[1].strip()
            cache["proposed_trade"] = [parse_proposed_trade(trade)]
            lines_to_pass["proposed_trade"] = l
        
        elif l.startswith("NEWLY PROPOSED TRADE:"):
            trade = l.split("NEWLY PROPOSED TRADE:")[1].strip()
            print(trade)
            cache["proposed_trade"] = [parse_proposed_trade(trade)]
            lines_to_pass["proposed_trade"] = l
        
        elif l.startswith("PLAYER RESPONSE: "):
            cache["player_response"] = [l.split("PLAYER RESPONSE: ")[1]]
            lines_to_pass["player_response"] = l
        else:
            print(f"..::UNPARSED: {l}::..")
            continue
        
    return lines_to_pass["proposed_trade"], lines_to_pass["player_response"], cache


In [142]:
conversations = {1: [{"role": "system", "content": structured_calls.format(potential_resources_txt, starting_resources_txt[1], goals_txt[1], n_rounds, roles[1])}],
               2: [{"role": "system", "content": structured_calls.format(potential_resources_txt, starting_resources_txt[2], goals_txt[2], n_rounds, roles[2])}]
                }


In [143]:
all_cache = {1: defaultdict(list), 2: defaultdict(list)}

chat = openai.ChatCompletion.create(model="gpt-4", messages=conversations[1],
    temperature=0, max_tokens=400,
)
conversations[1].append(dict(chat["choices"][0]["message"]))

response = chat["choices"][0]["message"]["content"]
trade_proposal, trade_decision, cache = parse_response(response)
all_cache[1].update(cache)


conversations[2].append({"role": "user", "content": trade_proposal})
chat2 = openai.ChatCompletion.create(model="gpt-4", messages=conversations[2],
    temperature=0, max_tokens=400,
)
conversations[2].append(dict(chat2["choices"][0]["message"]))
response2 = chat2["choices"][0]["message"]["content"]
trade_proposal2, trade_decision2, cache2 = parse_response(response2)
all_cache[2].update(cache2)
new_message = trade_decision2 + "\n" + trade_proposal2


conversations[1].append({"role": "user", "content": new_message})
chat3 = openai.ChatCompletion.create(model="gpt-4", messages=conversations[1],
    temperature=0, max_tokens=400,
)
conversations[1].append(dict(chat3["choices"][0]["message"]))
response3 = chat3["choices"][0]["message"]["content"]
trade_proposal3, trade_decision3, cache3 = parse_response(response3)
all_cache[1].update(cache3)


..::UNPARSED: ::..
..::UNPARSED: RESOURCES AFTER TRADE: X: 0, Y: 30, Z: 35::..
Player 1 Gives: Y: 15, Z: 5, Player 2 Gives: X: 15
..::UNPARSED: RESOURCES AFTER TRADE: X: 20, Y: 10, Z: 5::..
Player 1 Gives: X: 5, Player 2 Gives: Y: 5, Z: 5


In [144]:
conversations[1]

[{'role': 'system',
  'content': "You are playing a strategic game of trading resources with another player. You are Player 1. You are playing against a player whose resources you have no knowledge about.\n\nYou will respond one turn at a time. At each step, you will state your current resources and either propose or respond to a trade.\nIf it's your turn to propose, your response should be in the following format:\n\nRESOURCES: item1: amount, item2: amount, ... # Your resources\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\n\nand do not say anything else. If it's your turn to respond, your response should be in the following format:\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\nPLAYER RESPONSE: {ACCEPTED/REJECTED}\nRESOURCES AFTER TRADE: item1: amount, item2: amount, ... # Your resources\nNEWLY PROPOSED TRADE: Player 1 Gives: item1: amount,

In [135]:
conversations[1]

[{'role': 'system',
  'content': "You are playing a strategic game of trading resources with another player. You are Player 1. You are playing against a player whose resources you have no knowledge about.\n\nYou will respond one turn at a time. At each step, you will state your current resources and either propose or respond to a trade.\nIf it's your turn to propose, your response should be in the following format:\n\nRESOURCES: item1: amount, item2: amount, ... # Your resources\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\n\nand do not say anything else. If it's your turn to respond, your response should be in the following format:\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\nPLAYER RESPONSE: {ACCEPTED/REJECTED}\nRESOURCES AFTER TRADE: item1: amount, item2: amount, ... # Your resources\nNEWLY PROPOSED TRADE: Player 1 Gives: item1: amount,

In [106]:
print(response2), print(response)

PLAYER RESPONSE: ACCEPTED
RESOURCES AFTER TRADE: X: 0, Y: 30, Z: 35
NEWLY PROPOSED TRADE: Player 1 Gives: Y: 10, Z: 10, Player 2 Gives: X: 10
RESOURCES: X: 25, Y: 5

PROPOSED TRADE: Player 1 Gives: X: 5 Player 2 Gives: Y: 5, Z: 5


(None, None)

..::UNPARSED: RESOURCES AFTER TRADE: X: 0, Y: 30, Z: 35::..
Player 1 Gives: Y: 10, Z: 10, Player 2 Gives: X: 10


In [117]:
print(new_message)

PLAYER RESPONSE: ACCEPTED
NEWLY PROPOSED TRADE: Player 1 Gives: Y: 10, Z: 10, Player 2 Gives: X: 10


In [119]:
conversations

{1: [{'role': 'system',
   'content': "You are playing a strategic game of trading resources with another player. You are Player 1. You are playing against a player whose resources you have no knowledge about.\n\nYou will respond one turn at a time. At each step, you will state your current resources and either propose or respond to a trade.\nIf it's your turn to propose, your response should be in the following format:\n\nRESOURCES: item1: amount, item2: amount, ... # Your resources\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\n\nand do not say anything else. If it's your turn to respond, your response should be in the following format:\n\nPROPOSED TRADE: Player 1 Gives: item1: amount, item2: amount, ... Player 2 Gives: item1: amount, item2: amount, ...\nPLAYER RESPONSE: {ACCEPTED/REJECTED}\nRESOURCES AFTER TRADE: item1: amount, item2: amount, ... # Your resources\nNEWLY PROPOSED TRADE: Player 1 Gives: item1: am